In [1]:
import tensorflow as tf
import tensorflow_hub as hub

In [5]:
%cd /content/drive/MyDrive/RohithWorkspace/Object-Detection-Website
!git config --global user.email "007rohitjj04@gmail.com"
!git config --global user.name "Rohith04MVK"
!git add -A
!git commit -m "Still testing google colab"
!git push

/content/drive/MyDrive/RohithWorkspace/Object-Detection-Website
[main 84dad0b] Still testing google colab
 1 file changed, 1 insertion(+), 1 deletion(-)
 rewrite src/main.ipynb (70%)
Counting objects: 4, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 1.22 KiB | 311.00 KiB/s, done.
Total 4 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/Rohith04MVK/Object-Detection-Website.git
   5d77ccf..84dad0b  main -> main
